### WebCrawling
* 웹페이지의 종류
    * 정적페이지: 페이지의 데이터가 변경될 때 URL이 변경 O HTML 포멧의 데이터 수집
        * ex) 네이버 뉴스--> URL 바뀜
    * 동적페이지: 페이지의 데이터가 변경될 때 URL이 변경 X JSON 포멧의 데이터 수집
        * ex) 네이버 주식차트-->URL안바뀜
* Request Package
    * 브라우저의 URL을 입력하면 서버에서 데이터를 받아 화면에 출력:URL->DATA
    * Request 패키지: URL->DATA

### NAVER Stock Data
* Kospi 지숙
* kosdaq 지수
* USD: 원-달러 환율

In [1]:
import requests
import pandas as pd

In [2]:
# 1. 웹서비스를 분석: 크롬'개발자도구':URL을 획득가능
# https://m.stock.naver.com/

In [3]:
url = "https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=4"

In [4]:
# 2. Request->Respone:JSON(str)

In [5]:
response = requests.get(url)
response
# 여기서 200-->잘 받아왔다는 쏘리

<Response [200]>

In [6]:
response.text[:200]

'[{"localTradedAt":"2022-06-23","closePrice":"2,314.32","compareToPreviousClosePrice":"-28.49","compareToPreviousPrice":{"code":"5","text":"하락","name":"FALLING"},"fluctuationsRatio":"-1.22","openPrice"'

In [7]:
# 3. JSON(str) ->List,dict -> DataFrame

In [8]:
data = response.json()

type(data),data[:2]

(list,
 [{'localTradedAt': '2022-06-23',
   'closePrice': '2,314.32',
   'compareToPreviousClosePrice': '-28.49',
   'compareToPreviousPrice': {'code': '5', 'text': '하락', 'name': 'FALLING'},
   'fluctuationsRatio': '-1.22',
   'openPrice': '2,347.84',
   'highPrice': '2,361.23',
   'lowPrice': '2,306.48'},
  {'localTradedAt': '2022-06-22',
   'closePrice': '2,342.81',
   'compareToPreviousClosePrice': '-66.12',
   'compareToPreviousPrice': {'code': '5', 'text': '하락', 'name': 'FALLING'},
   'fluctuationsRatio': '-2.74',
   'openPrice': '2,417.11',
   'highPrice': '2,418.05',
   'lowPrice': '2,342.81'}])

In [9]:
df = pd.DataFrame(data)[['localTradedAt','closePrice']]
df.tail(2)

,localTradedAt,closePrice
8,2022-06-13,"2,504.51"
9,2022-06-10,"2,595.87"


In [10]:
# 4.함수 만들기
# params: pagesize,page

In [11]:
def stock_price(pagesize,page,code="KOSPI"):
    """This function is crawling stock prcie from naver webpage:
    Params
    ------
    pagesize : int : one page size
    page : int : page number
    code : str : KOSPI OR KOSDAQ
    
    
    Return
    ------
    type: DataFrame :display data, price columns
    """
    url = f"https://m.stock.naver.com/api/index/{code}/price?pageSize={pagesize}&page={page}"
    #f는 포멧팅을 위한 언어로써 {}를 사용하기 위해서는 필요함
    response = requests.get(url)
    data= response.json()
    return pd.DataFrame(data)[['localTradedAt','closePrice']]

In [12]:
stock_price(30,5)

,localTradedAt,closePrice
0,2022-02-10,"2,771.93"
1,2022-02-09,"2,768.85"
2,2022-02-08,"2,746.47"
3,2022-02-07,"2,745.06"
4,2022-02-04,"2,750.26"
5,2022-02-03,"2,707.82"
6,2022-01-28,"2,663.34"
7,2022-01-27,"2,614.49"
8,2022-01-26,"2,709.24"
9,2022-01-25,"2,720.39"


In [13]:
# Kosdaq 데이터 수집 코드 작성

In [14]:
def kosdaq_price(pagesize,page):
    url=f"https://m.stock.naver.com/api/index/KOSDAQ/price?page={pagesize}&pageSize={page}"
    response =requests.get(url)
    data = response.json()
    return pd.DataFrame(data)[['localTradedAt','closePrice']]

In [15]:
kosdaq_price(10,5)

,localTradedAt,closePrice
0,2022-05-31,893.36
1,2022-05-30,886.44
2,2022-05-27,873.97
3,2022-05-26,871.43
4,2022-05-25,872.69


In [16]:
kospi = stock_price(60,1,"KOSPI")
kosdaq = stock_price(60,1,"KOSDAQ")

In [17]:
# docstring: 함수를 사용하는 방법을 문자열로 작성
# help(),shift+tab

help(stock_price)

Help on function stock_price in module __main__:

stock_price(pagesize, page, code='KOSPI')
    This function is crawling stock prcie from naver webpage:
    Params
    ------
    pagesize : int : one page size
    page : int : page number
    code : str : KOSPI OR KOSDAQ
    
    
    Return
    ------
    type: DataFrame :display data, price columns



In [18]:
### 최근 60일치 원달러 환율 데이터 수집


In [19]:
url ="https://api.stock.naver.com/marketindex/exchange/FX_USDKRW/prices?page=1&pageSize=60"

In [28]:
response = requests.get(url)
data = response.json()
usd = pd.DataFrame(data)[['localTradedAt','closePrice']]
usd.tail(2)

,localTradedAt,closePrice
58,2022-05-12,"1,290.50"
59,2022-05-11,"1,274.50"


In [21]:
kospi.tail(2)

,localTradedAt,closePrice
58,2022-05-12,"2,550.08"
59,2022-05-11,"2,592.27"


In [22]:
kosdaq.tail(2)

,localTradedAt,closePrice
58,2022-05-12,833.66
59,2022-05-11,866.34


In [23]:
def ExRate(pagesize,page,won):
#     """This function is crawling stock prcie from naver webpage:
#     Params
#     ------
#     pagesize : int : one page size
#     page : int : page number
#     Won : str : USD, EUR,JPY,CNY
#     Return
#     ------
#     type: DataFrame :display data, price columns
#     """
    url = f"https://api.stock.naver.com/marketindex/exchange/FX_{won}KRW/prices?page={pagesize}&pageSize={page}"
    response = requests.get(url)
    data= response.json()
    return pd.DataFrame(data)[['localTradedAt','closePrice']]

In [24]:
eur = ExRate(60,1,'EUR')

# 데이터 분석
* 상관 관계 분석: 두 데이터 집합 사이에 어떤 관계가 있는 확인하는 분석방법
* 원 달러 환율이 높으면 코스피,코스닥 지수가 낮다.-->음의 상관관계를 갖는다

#### 8. 상관관계 분석
-피어슨 상관계수(Pearson Correlation Coefficient)
- 두 데이터 집합의 상관도를 분석할때 사용되는 지표
- 상관계수의 해석
    - -1에 가까울수록 서로 반대방향으로 움직임
    - 1에 가까울수록 서로 같은방향으로 움직임
    - 0에 가까울수록 두 데이터는 관계가 없음

In [25]:
# 피어슨 상관계수
# 1과 가까울수록 강한 양의 상관관계
# -1과 가까울수록 강한 음의 상관관계
# 0과 가까울수록 관계가 없다

In [30]:
#데이터 전처리
df = kospi.copy()
df["kosdaq"]=kosdaq["closePrice"]
df["usd"]=usd["closePrice"]
df = df.rename(columns={
    "closePrice" : "kospi"
})
df

,localTradedAt,kospi,kosdaq,usd
0,2022-08-04,"2,474.64",823.69,"1,309.30"
1,2022-08-03,"2,461.45",815.36,"1,312.00"
2,2022-08-02,"2,439.62",804.34,"1,312.00"
3,2022-08-01,"2,452.25",807.61,"1,305.00"
4,2022-07-29,"2,451.50",803.62,"1,304.50"
5,2022-07-28,"2,435.27",798.32,"1,302.00"
6,2022-07-27,"2,415.53",795.70,"1,314.00"
7,2022-07-26,"2,412.96",789.93,"1,311.00"
8,2022-07-25,"2,403.69",789.69,"1,312.00"
9,2022-07-22,"2,393.14",789.75,"1,310.00"


In [ ]:
# 컬럼의 데이터 타입 변경:str >float
# df[column].apply():모든 데이터를 함수에 대입한 결과를 출력
#lambda

df.dtypes

In [ ]:
df['kospi']=df['kospi'].apply(lambda data:float(data.replace(",","")))
df['kosdaq']=df['kosdaq'].apply(lambda data:float(data.replace(",","")))
df['usd']=df['usd'].apply(lambda data:float(data.replace(",","")))

In [ ]:
df.dtypes

In [ ]:
df[['kospi','kosdaq','usd']].corr()

In [ ]:
#kospi-kosdaq:0.984 1에 가까우

In [ ]:
data1 = [1,2,3]
data2= data1 #주소값 복사:얉은 복사
data3 = data1.copy() #값 복사 :깊은 복사
print(data1,data2,data3)
data1[1]=4
print(data1,data2,data3)

In [ ]:
#apply(func):모든 데이터를 function을 적용시킨 결과 출력

In [ ]:
df = pd.DataFrame([
    {"age":23},
    {"age":36},
    {"age":27}
])
df

In [ ]:
def change_age(age):
    return age // 10*10
#연령대로 나누어주는 func

In [ ]:
df['ages']=df['age'].apply(change_age)
df

In [ ]:
#lambda: 일회성 함수
#사용이유: 간단한 함수를 메모리를 절약하여 사용

In [ ]:
#함수 3개 사용, 메모리 3칸사용
def plus(n1,n2):
    return n1+n2
def minus(n1,n2):
    return n1-n2
def calc(func,n1,n2):
    return func(n1,n2)

In [ ]:
calc(plus,1,2), calc(minus,1,2)

In [ ]:
#lambda 변수:return값
plus_lambda = lambda n1,n2: n1+n2
plus(2,3),plus_lambda(2,3)

In [ ]:
#함수 1개사용 메모리 1칸사용
def calc(func,n1,n2):
    return func(n1,n2)
calc(lambda n1,n2:n1+n2,1,2), calc(lambda n1,n2:n1-n2,1,2)